# Applied Data Science Capstone - Week 3 Project

## Part 1: Data Extraction from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Using Pandas and Beautifulsoup

In [1]:
# Install BeautifulSoup and tabulate, if you have not already done so
# !pip install bs4
# !pip install tabulate

### import the necessary lib

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

### Accessing the wiki page

In [15]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

# Use BeautifulSoup to part the page content based on lxml
soup = BeautifulSoup(res.content,'lxml')

# Use BeatifulSoup to find the table in the page
table = soup.find_all('table')[0] 

# Content the table content in the html to a Dataframe
neighborhood_list = pd.read_html(str(table))
df_can_neighborhood = pd.DataFrame(neighborhood_list[0])
df_can_neighborhood.head()
# print( tabulate(df_can_neighborhood[0], headers='keys', tablefmt='psql') )

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Rename the Postcode column to PostalCode

In [16]:
df_can_neighborhood.rename(columns = {"Postcode" : "PostalCode", "Neighbourhood" : "Neighborhood"}, inplace = True)
df_can_neighborhood.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop Borough that are Not assigned

In [17]:
indexNames = df_can_neighborhood[ df_can_neighborhood['Borough'] == "Not assigned"].index
df_can_neighborhood.drop(indexNames, inplace = True)
df_can_neighborhood.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Sort the Dataset by PostalCode

In [18]:
df_can_neighborhood.sort_values(by = 'PostalCode', inplace = True)
df_can_neighborhood.head()

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
28,M1C,Scarborough,Port Union
27,M1C,Scarborough,Rouge Hill
26,M1C,Scarborough,Highland Creek


### If neighborhood is Not Assigned, as it value to be same as Borough. There are NO neighborhood that are Not assigned

In [19]:
df_can_neighborhood[df_can_neighborhood['Neighborhood'] == 'Not Assigned']

,PostalCode,Borough,Neighborhood


### Postal 'M5A' is NOT defined twice as mentioned in Assignment notes

In [20]:
df_can_neighborhood[df_can_neighborhood['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront


### Combine neighbor that has the same PostalCode

In [21]:
df_can_neighborhood['Neighborhood'] = df_can_neighborhood[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode'], as_index = False)['Neighborhood'].transform(lambda x: ','.join(x))
df_can_neighborhood.drop_duplicates(subset='PostalCode', inplace = True)
df_can_neighborhood.head()

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,"Rouge,Malvern"
28,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
41,M1E,Scarborough,"Guildwood,Morningside,West Hill"
52,M1G,Scarborough,Woburn
61,M1H,Scarborough,Cedarbrae


In [22]:
df_can_neighborhood

,PostalCode,Borough,Neighborhood
10,M1B,Scarborough,"Rouge,Malvern"
28,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
41,M1E,Scarborough,"Guildwood,Morningside,West Hill"
52,M1G,Scarborough,Woburn
61,M1H,Scarborough,Cedarbrae
75,M1J,Scarborough,Scarborough Village
90,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
107,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
122,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
140,M1N,Scarborough,"Cliffside West,Birch Cliff"


### Print the number of rows in the DataSet

In [23]:
print('The number of row in the Dataset = ', df_can_neighborhood.shape[0])

The number of row in the Dataset =  103


## Part 2. Get Lat and Long based on Postal Code

### Import the necessary lib

In [ ]:
# Install the library, if you have already done so
!pip install geocoder

In [24]:
import geocoder

In [29]:
# Create a PostalCode list
postal_list = df_can_neighborhood['PostalCode']
postal_list
type(postal_list)

pandas.core.series.Series

### Retrieve lat and long based on Postal Code

In [28]:
# Try to retrieve a single PostalCode
postal_code = 'M5A'

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  # g = geocoder.google('{}, Toronto, Ontario'.format(postal_code), key = 'xxxx')
  g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code), key = 'xxxx')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print("Postal Code = {}, Lat = {}, Long = {}".format(postal_code, latitude, longitude))

Postal Code = M5A, Lat = 43.65029500000003, Long = -79.35916572299999


In [48]:
# Retrieve lat and long for the postal code list of Toronto
postal_latlng = []

for postal_code in postal_list.iteritems():
    # print('postal_code = ', postal_code)

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code), key = 'xxxxx')
        lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]

        postal_latlng.append([postal_code[1], latitude, longitude])
        # print("Postal Code = {}, Lat = {}, Long = {}".format(postal_code, latitude, longitude))    

postal_latlng

[['M1B', 43.811525000000074, -79.19551746399998],
 ['M1C', 43.78566500000005, -79.15872457299997],
 ['M1E', 43.76581500000003, -79.17519294699997],
 ['M1G', 43.768369121000035, -79.21758999999997],
 ['M1H', 43.76968799900004, -79.23943999999995],
 ['M1J', 43.74312500000008, -79.23174973599998],
 ['M1K', 43.72627568400003, -79.26362499999993],
 ['M1L', 43.71305350000006, -79.28505499999994],
 ['M1M', 43.724234575000025, -79.22792499999997],
 ['M1N', 43.69677000000007, -79.25996735299998],
 ['M1P', 43.759975000000054, -79.26897418299995],
 ['M1R', 43.750710464000065, -79.30055999999996],
 ['M1S', 43.79394000000008, -79.26797613999997],
 ['M1T', 43.78472500000004, -79.29906603299997],
 ['M1V', 43.81768500000004, -79.28018721399997],
 ['M1W', 43.80088292800008, -79.32073999999994],
 ['M1X', 43.83421500000003, -79.21670085099998],
 ['M2H', 43.80284500000005, -79.35620744999994],
 ['M2J', 43.780970000000025, -79.34781328099996],
 ['M2K', 43.781015000000025, -79.38052867199997],
 ['M2L', 43.7

### Merge the Lat and Long to the existing Dataset based on Postal Code

In [60]:
# Convert postal_latlng to Dataframe
df_postal_latlng = pd.DataFrame(postal_latlng)
df_postal_latlng.columns = ['PostalCode', 'Latitude', 'Longitude']
df_postal_latlng.head()

df_toronto = pd.merge(df_can_neighborhood, df_postal_latlng, on = 'PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [61]:
print('The number of row in the Dataset = ', df_toronto.shape[0])

The number of row in the Dataset =  103
